# Training and testing the model

* Open all the datasets

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime as dt 

In [9]:
Df = pd.read_csv('./DataSets/PreProcessed_EnergyConsumption_WeatherData.csv', header=0)
Df.head()

,Time,Energy Usage,t,tp,rf,wr,wg,wsr,wsg,regen,ldred,ldstat,sonne
0,2021-01-27 11:00:00,8.03,0.0,-3.8,75.0,130.0,3.6,NaN,7.6,0.0,1018.4,973.5,93.0
1,2021-01-27 12:00:00,6.85,1.9,-3.3,68.0,150.0,5.4,NaN,9.4,0.0,1017.7,973.1,66.0
2,2021-01-27 13:00:00,8.86,3.5,-2.8,63.0,190.0,7.6,NaN,11.2,0.0,1016.8,972.5,100.0
3,2021-01-27 14:00:00,7.37,4.4,-5.3,49.0,120.0,9.4,NaN,14.8,0.0,1016.3,972.1,99.0
4,2021-01-27 15:00:00,7.67,5.0,-7.6,40.0,150.0,5.4,NaN,11.2,0.0,1015.5,971.5,100.0


In [10]:
DfH = pd.read_csv('./DataSets/EnergyReading_Days_Dataset.csv', header=0)
DfH.head()

,Time,Holiday,Week Day,Energy Usage,cos_x,sin_x
0,2021-01-27 11:00:00,0.0,2,8.03,-0.222521,0.974928
1,2021-01-27 12:00:00,0.0,2,6.85,-0.222521,0.974928
2,2021-01-27 13:00:00,0.0,2,8.86,-0.222521,0.974928
3,2021-01-27 14:00:00,0.0,2,7.37,-0.222521,0.974928
4,2021-01-27 15:00:00,0.0,2,7.67,-0.222521,0.974928


### Merge the holidays and week day decoding 

In [11]:
Df.insert(1,'Holiday',DfH["Holiday"])
Df.insert(1,'Cos',DfH["cos_x"])
Df.insert(1,'Sin',DfH["sin_x"])
Df.insert(1,'Week Day',DfH["Week Day"])

In [16]:
Df.reindex(columns=["Time","Week Day","Sin","Cos","Holiday","t","tp","Energy Usage"],inplace=True)

,Time,Week Day,Sin,Cos,Holiday,t,tp,Energy Usage
0,NaN,2,0.974928,-0.222521,0.0,0.0,-3.8,8.03
1,NaN,2,0.974928,-0.222521,0.0,1.9,-3.3,6.85
2,NaN,2,0.974928,-0.222521,0.0,3.5,-2.8,8.86
3,NaN,2,0.974928,-0.222521,0.0,4.4,-5.3,7.37
4,NaN,2,0.974928,-0.222521,0.0,5.0,-7.6,7.67
...,...,...,...,...,...,...,...,...
6721,NaN,2,0.974928,-0.222521,0.0,5.7,4.7,6.68
6722,NaN,2,0.974928,-0.222521,0.0,6.7,5.2,7.26
6723,NaN,2,0.974928,-0.222521,0.0,7.3,5.9,6.81
6724,NaN,2,0.974928,-0.222521,0.0,7.4,6.4,8.81


### Splitting the Time 

In [13]:
from datetime import datetime as dt 
def SplitTime(data,col): 
    
    year = [] 
    month = [] 
    day = [] 
    hour = [] 
    
    for str_date in data[col]: 
        dti = dt.strptime(str_date, "%Y-%m-%d %H:%M:%S")
        year.append(dti.year)
        month.append(dti.month)
        day.append(dti.day)
        hour.append(dti.hour)
   
    #Delete the old column
    data.drop(col, inplace = True, axis=1) 

    #Insert the newone
    data.insert(0, 'year',year)
    data.insert(1, 'month',month)
    data.insert(2, 'day',day)
    data.insert(3, 'hour',hour)

    return data

CDf= SplitTime(Df,'Time')

In [17]:
Df

,year,month,day,hour,Week Day,Sin,Cos,Holiday,Energy Usage,t,tp,rf,wr,wg,wsr,wsg,regen,ldred,ldstat,sonne
0,2021,1,27,11,2,0.974928,-0.222521,0.0,8.03,0.0,-3.8,75.0,130.0,3.6,NaN,7.6,0.0,1018.4,973.5,93.0
1,2021,1,27,12,2,0.974928,-0.222521,0.0,6.85,1.9,-3.3,68.0,150.0,5.4,NaN,9.4,0.0,1017.7,973.1,66.0
2,2021,1,27,13,2,0.974928,-0.222521,0.0,8.86,3.5,-2.8,63.0,190.0,7.6,NaN,11.2,0.0,1016.8,972.5,100.0
3,2021,1,27,14,2,0.974928,-0.222521,0.0,7.37,4.4,-5.3,49.0,120.0,9.4,NaN,14.8,0.0,1016.3,972.1,99.0
4,2021,1,27,15,2,0.974928,-0.222521,0.0,7.67,5.0,-7.6,40.0,150.0,5.4,NaN,11.2,0.0,1015.5,971.5,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6721,2021,11,3,12,2,0.974928,-0.222521,0.0,6.68,5.7,4.7,93.0,140.0,5.4,NaN,11.2,0.0,1008.5,964.9,0.0
6722,2021,11,3,13,2,0.974928,-0.222521,0.0,7.26,6.7,5.2,90.0,70.0,5.4,NaN,13.0,0.0,1007.7,964.3,0.0
6723,2021,11,3,14,2,0.974928,-0.222521,0.0,6.81,7.3,5.9,91.0,80.0,3.6,NaN,9.4,0.0,1006.8,963.6,0.0
6724,2021,11,3,15,2,0.974928,-0.222521,0.0,8.81,7.4,6.4,93.0,300.0,3.6,NaN,5.4,0.0,1006.7,963.5,0.0


## Linear Regression Model 